# <center>  Enhancer Pixel Sequence  Deep Convolutional Generative Adversarial Network </center>

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1],'GPU')

#tf.config.experimental.set_memory_growth(gpus[2],True)

In [ ]:

for device in gpus:
    tf.config.experimental.set_memory_growth(device, True)

# Importing Libararies

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import os
import tqdm 
from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, LeakyReLU, Conv1D, Conv2D, Conv2DTranspose, Flatten
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import time

In [ ]:

print(tf.test.is_built_with_cuda())

In [ ]:
#

In [ ]:
gpus

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
print(tf.__version__)
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

# Defining the variables for the ease of use in the code

In [ ]:
pic_dir = './images/'
images_count = 32693
width = 28
height = 28
images = []
x1 =  0
y1 = 0
x2 = 152
y2 = 104

In [ ]:
len(os.listdir(pic_dir))

In [ ]:
for pic_file in tqdm.tqdm(os.listdir(pic_dir)[:images_count]):
    # Saving the aspect ratio and properties of images using thumbnail and antialias
    pic = Image.open(pic_dir + pic_file)
    images.append(img_to_array(pic))

# Converting the images into num

In [ ]:
# Converting images into numpy and normalizing the vals
images = ((np.array(images)-127.5)/ 127.5)
print("Images Normalized...")

In [ ]:
# Confirming the size
images[0].shape

In [ ]:
def create_batch(x_train): 
    batch_size = 30
    dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000) 
    # Combines consecutive elements of this dataset into batches. 
    dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(1) 
    # Creates a Dataset that prefetches elements from this dataset 
    return dataset

In [ ]:
dataset = create_batch(images)

# Defining the Generator side of the DCGAN

In [ ]:
latent_vector = 100
channel = 1
def generator_model():
    # Flattened and noise vector
    generator_input = keras.Input(shape = (latent_vector,))
    
    # Starting from a Dense layer
    x = Dense(100 * 28 * 28)(generator_input)
    x = LeakyReLU()(x)
    
    # Reshaping to organize 
    x = Reshape((28, 28, 100))(x)
    
    # Applying convlution operation to get some initial features
    # Applying 155 filters with a kernel size of 5
    # We dont add padding to make it of equivalent size as input
    x = Conv2D(filters = 256, kernel_size = 6, strides=4, padding="same")(x)
    x = LeakyReLU()(x)
    
    
    # Applying the transpose operation so that when the conv is applied we get matrix of higher dimensions i-e. https://www.machinecurve.com/index.php/2019/09/29/understanding-transposed-convolutions/
    # https://distill.pub/2016/deconv-checkerboard/
    x = Conv2DTranspose(filters = 256, kernel_size = 2, strides=2, padding = "same")(x)
    x = LeakyReLU()(x)
    
    
    # Conv2DTrans num 2
    x = Conv2DTranspose(filters = 256, kernel_size = 4, strides=2, padding = "same")(x)
    x = LeakyReLU()(x)
    
    x = Conv2D(channel, kernel_size = 7, activation='tanh', padding='same')(x)
    
    
 
    
    '''  
       # Conv2DTrans num 3
    x = Conv2DTranspose(filters = 256, kernel_size = 4, strides=2, padding = "same")(x)
    x = LeakyReLU()(x)
    
    x = Conv2D(filters = 512, kernel_size = 4, padding="same" )(x)
    x = LeakyReLU()(x)
    
    x = Conv2D(filters = 512, kernel_size = 6, padding="same" )(x)
    x = LeakyReLU()(x)
    
    
    
    '''
    generator = Model(generator_input, x)
    
    return generator

In [ ]:
generator=generator_model()
generator.summary()

# Defining the Discriminator side of the DCGAN

In [ ]:
def create_discriminator():
    
    # Defining the shape of the input that is to be given as input
    discriminator_input = keras.Input(shape = (28, 28, 1))
    
    # Defining the convolution layer in a way so that the size is reduced to its half
    x = Conv2D(filters = 228, kernel_size = 2, strides = 2)(discriminator_input)
    x = LeakyReLU()(x)
    
    # conv layer 2
    x = Conv2D(filters=104, kernel_size=2, strides=2)(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.3)(x)
    
    # conv layer 3
    #x = Conv2D(filters=52, kernel_size=2, strides=2)(x)
    #x = LeakyReLU()(x)
    
    # conv layer 4
    # x = Conv2D(filters=26, kernel_size=2, strides=2)(x)
    #x = LeakyReLU()(x)
    
    # Flattening the matrixes
    x = Flatten()(x)
    
    # Performing the regularization to minimize overfitting
    x = Dropout(rate=0.3)(x)
    
    # Defining the output from the discriminator which is always the probability
    x = Dense(units=1, activation='sigmoid')(x)
    
    # From a tensor to a complete model
    discriminator = Model(discriminator_input , x )
    '''
    # Defining the optimizer for the Discriminator
    optimizer = tf.keras.optimizers.RMSprop(
        rho=0.9,
        momentum=0.0,
        epsilon=1e-07,
        centered=False,
        learning_rate=0.001,
        clipvalue=1.0
    )
    
    # Compiling the model
    discriminator.compile(
        optimizer = optimizer,
        loss = keras.losses.binary_crossentropy
    )
    '''
    return discriminator

In [ ]:
discriminator=create_discriminator()
discriminator.summary()

# Creating the image using the untrained Generator

**Using the (as yet untrained) generator to create an image.**

In [ ]:
# Creating random vector of size 100 from Normal Distribution
noise = tf.random.normal([1,100])
print("tensor created with the random values of size 100 ...")

# No training has been specified
generated_image = generator(noise, training=False)
plt.figure(1, figsize=(10, 10))
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

# Using untrained discriminator to classify the generated images as real or fake

**The model will be trained to output positive values for real images, and negative values for fake images.**

In [ ]:
decision = discriminator(generated_image)
print(decision)

# Compiling and combining phase

In [ ]:
# compile discriminator using binary cross entropy loss and adam optimizer 
discriminator.compile(loss ="binary_crossentropy", optimizer ="adam") 
# make  discriminator no-trainable as of  now 
discriminator.trainable = False

In [ ]:
# Combine both generator and discriminator 
gan = keras.models.Sequential([generator, discriminator])
# compile generator using binary cross entropy loss and adam optimizer
gan.compile(loss="binary_crossentropy", optimizer="adam")

# Defining the Training 

In [ ]:
epochs = 500
# We have already defined latent_vector ppreviously
batch_size = 30
# Creating the random vector from noise 
total_vec_seed = tf.random.normal(shape=[batch_size, latent_vector])


In [ ]:
total_vec_seed

In [ ]:
def train_dcgan(gan, images, batch_size, num_features, epochs=3):
    generator, discriminator = gan.layers
    for epoch in tqdm.tqdm(range(epochs)):
        print()
        print("Epoch {}/{}".format(epoch + 1, epochs))
        for X_batch in images:
            # Create a random noise of size batch * 100
            noise = tf.random.normal(shape=[batch_size, num_features])
            generated_images = generator(noise)
           
            # Combining the real and fake images to train the discriminator
            fake_and_real = tf.concat((generated_images, X_batch), axis=0)
            
            # Providing batch of real and fake images to the discrimnator
            # training the discriminator using it
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable=True
            discriminator.train_on_batch(fake_and_real, y1)
            # Training the GAN
            # Providing the generated images and fooling the discriminator with fake labels of 1
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable=False
            gan.train_on_batch(noise, y2)
        
            # Generate and save images
            #generate_and_save_images(generator, epoch+1, noise)
            
        print(discriminator.evaluate(fake_and_real, y1))
        
        
        generate_and_save_images(generator, epoch, noise)

# Defining and saving Image Generation Via Generator

In [ ]:
preds_to =list()
preds_for=list()
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training = False)
    preds_to.append(predictions)
    fig = plt.figure(figsize=(10, 10))
    
    for i in range(5):
        plt.subplot(1, 5, i+1)
        preds_to_show = predictions[i, :, :] * 127.5 + 127.5
        
        #preds_to_show = np.squeeze(preds_to_show)
        plt.imshow(preds_to_show, cmap='gray')
        plt.axis('off')
        
    
    plt.savefig('image_epoch_{:04d}.png'.format(epoch))
    #print("Before norm",preds_to)
    #print("After norm", preds_for)
    

In [ ]:
train_dcgan(gan, dataset, batch_size, latent_vector, epochs=5000)